##### doc2vec

In [1]:
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "Malgun Gothic"
plt.rcParams["axes.unicode_minus"] = False

import numpy as np
import pandas as pd
import re
from konlpy.tag import Okt
from tqdm import tqdm

In [2]:
data = pd.read_csv("data_토큰화(doc,token,text_noun).csv", encoding='utf-8-sig')
data.head()

,Unnamed: 0,회사,doc,token,text_noun
0,0,네이버,커리어 경력 쌓고 싶은 사람에게 추천 수평적 사무실 분위기와 복지가 다른 곳 보다는...,"['커리어', '경력', '사람', '추천', '수평', '사무실', '분위기', ...","['커리어', '경력', '사람', '추천', '수평', '사무실', '분위기', ..."
1,1,네이버,자유로운 복장 분위기가 일단 편해서 좋았어요 물론 업무는 당연히 강도가 있어야 할 ...,"['자유롭다', '복장', '분위기', '편하다', '좋다', '업무', '당연하다...","['복장', '분위기', '업무', '강도', '부분', '분위기', '근무', '..."
2,2,네이버,워라밸과 성장을 동시에 챙길 수 있는 몇 안되는 기업 앞으로도 가장 전망이 좋은 플...,"['워', '밸', '성장', '동시', '수', '안되다', '기업', '앞', ...","['워', '밸', '성장', '동시', '수', '기업', '앞', '가장', '..."
3,3,네이버,개발자가 영향력을 좀 발휘하며 일할수 있는곳 it업계 1위 개발에만 집중하며 일할 ...,"['개발자', '영향력', '발휘', '곳', '업계', '위', '개발', '집중...","['개발자', '영향력', '발휘', '곳', '업계', '위', '개발', '집중..."
4,4,네이버,아르바이트 생이었지만 꿈의 직장이란게 이런 것이구나 느낄 수 있었다 휴가 연차 등 ...,"['아르바이트', '생', '꿈', '직장', '이란', '이렇다', '수', '휴...","['아르바이트', '생', '꿈', '직장', '이란', '수', '휴가', '연차..."


In [3]:
#필요없는 컬럼 삭제
del data['Unnamed: 0']
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60573 entries, 0 to 60572
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   회사         60573 non-null  object
 1   doc        60573 non-null  object
 2   token      60573 non-null  object
 3   text_noun  60573 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


In [4]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.cluster import KMeans
from gensim.test.utils import common_texts

In [6]:
import ast
token = []
token_noun = []
for i in range(len(data['token'])):
    token.append(ast.literal_eval(data['token'][i]))
    token_noun.append(ast.literal_eval(data['text_noun'][i]))

In [7]:
data['token'] = token
data['text_noun'] = token_noun

In [8]:
documents = [TaggedDocument(word, [i]) for i, word in enumerate(data['token'])]

In [9]:
max_epochs = 10
vec_size = 20
alpha = 0.025

model = Doc2Vec(vector_size=20,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=3,
                dm =1)
  
model.build_vocab(documents)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(documents,
                total_examples=model.corpus_count,
                epochs=10)
    # decrease the learning rate
    model.alpha -= 0.002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9


In [10]:
data['token'][0]

['커리어',
 '경력',
 '사람',
 '추천',
 '수평',
 '사무실',
 '분위기',
 '복지',
 '곳',
 '편',
 '회사',
 '시스템',
 '비',
 '효율',
 '부분',
 '업',
 '무량',
 '많다',
 '편이',
 '그냥',
 '간단하다',
 '더',
 '소통',
 '좋다']

In [11]:
vector = model.infer_vector(data['token'][0])

In [12]:
len(data['token'])

60573

In [13]:
model.random.seed(9999)
rv = []
for i in tqdm(data['token']):
    vector = model.infer_vector(i)
    rv.append(vector)

100%|██████████| 60573/60573 [00:18<00:00, 3232.26it/s]


In [14]:
len(rv)

60573

In [15]:
data['review_vec'] = rv

In [16]:
data.rename(columns = {'Unnamed: 0': 'doc_num'}, inplace=True)

In [19]:
#data.drop(['Unnamed: 0.3','Unnamed: 0.2','Unnamed: 0.1'], axis=1, inplace=True)

In [18]:
data

,회사,doc,token,text_noun,review_vec
0,네이버,커리어 경력 쌓고 싶은 사람에게 추천 수평적 사무실 분위기와 복지가 다른 곳 보다는...,"[커리어, 경력, 사람, 추천, 수평, 사무실, 분위기, 복지, 곳, 편, 회사, ...","[커리어, 경력, 사람, 추천, 수평, 사무실, 분위기, 복지, 곳, 편, 회사, ...","[-0.3265629, -0.20540974, 0.1888582, 0.1315793..."
1,네이버,자유로운 복장 분위기가 일단 편해서 좋았어요 물론 업무는 당연히 강도가 있어야 할 ...,"[자유롭다, 복장, 분위기, 편하다, 좋다, 업무, 당연하다, 강도, 부분, 편하다...","[복장, 분위기, 업무, 강도, 부분, 분위기, 근무, 환경, 중요시, 생각, 물리...","[-0.4096759, 0.10019315, -0.03808209, -0.01951..."
2,네이버,워라밸과 성장을 동시에 챙길 수 있는 몇 안되는 기업 앞으로도 가장 전망이 좋은 플...,"[워, 밸, 성장, 동시, 수, 안되다, 기업, 앞, 가장, 전망, 좋다, 플랫폼,...","[워, 밸, 성장, 동시, 수, 기업, 앞, 가장, 전망, 플랫폼, 임밸류, 문화,...","[-0.36782032, -0.23058303, 0.074808866, -0.011..."
3,네이버,개발자가 영향력을 좀 발휘하며 일할수 있는곳 it업계 1위 개발에만 집중하며 일할 ...,"[개발자, 영향력, 발휘, 곳, 업계, 위, 개발, 집중, 수, 보상, 복지, 좋다...","[개발자, 영향력, 발휘, 곳, 업계, 위, 개발, 집중, 수, 보상, 복지, 케바...","[-0.33013007, -0.117650464, 0.10383661, -0.015..."
4,네이버,아르바이트 생이었지만 꿈의 직장이란게 이런 것이구나 느낄 수 있었다 휴가 연차 등 ...,"[아르바이트, 생, 꿈, 직장, 이란, 이렇다, 수, 휴가, 연차, 복지, 눈치, ...","[아르바이트, 생, 꿈, 직장, 이란, 수, 휴가, 연차, 복지, 눈치, 전혀, 자...","[-0.22602941, -0.14391607, 0.14934903, 0.08710..."
...,...,...,...,...,...
60568,여기어때,돈은 있다 성장을 원한다 높은 보상을 원한다면 도전해보라 조직원 평균연령이 젊음(그...,"[돈, 성장, 원하다, 높다, 보상, 원하다, 도전, 직원, 평균연령, 젊음, 사고...","[돈, 성장, 보상, 도전, 직원, 평균연령, 젊음, 사고, 분위기, 자체, 열정,...","[-0.43693778, -0.34135696, 0.12423124, -0.0626..."
60569,여기어때,자율적인 분위기 깨어있는 조직문화 안정적인 회사를 원하면 지원해도됨 경영진의 마인드...,"[자율, 분위기, 조직, 문화, 안정, 회사, 원하다, 지원, 경영, 진의, 마인드...","[자율, 분위기, 조직, 문화, 안정, 회사, 지원, 경영, 진의, 마인드, 후, ...","[-0.42883778, -0.39161068, 0.2860069, 0.329696..."
60570,여기어때,BEST 일에 대한 열정이 있고 대인관계에 자신이 있는 자는 쉽게 높은 연봉과 직급...,"[대한, 열정, 대인관계, 높다, 연봉, 직급, 수, 젊다, 분위기, 분위기, 좋다...","[대한, 열정, 대인관계, 연봉, 직급, 수, 분위기, 분위기, 거, 인사, 시스템...","[-0.29673645, -0.22056107, 0.20917991, 0.06296..."
60571,여기어때,젊은 분위기 야근 많고 주말근무도 많음 월급은 적음 작은 규모이지만 비전을 가지고 ...,"[젊다, 분위기, 야근, 많다, 주말, 근무, 많다, 월급, 작다, 규모, 가지, ...","[분위기, 야근, 주말, 근무, 월급, 규모, 가지, 움직, 미래, 사장, 마인드,...","[-0.33322313, -0.300865, 0.05103625, -0.124771..."


In [20]:
data.to_pickle('data_벡터화(토큰).pkl')
data.to_csv("data_벡터화(토큰).csv", encoding='utf-8-sig')